```
1. 사전 준비 사항
Docker Desktop 설치 및 실행

Docker 공식 사이트에서 설치 후 실행.

Docker가 Linux 컨테이너 모드인지 확인 (Settings > General > Use the WSL 2 based engine 체크).

Docker Compose 확인

docker-compose는 Docker Desktop에 기본 포함되어 있음.

터미널 (PowerShell or WSL)에서 버전 확인:

sh
docker-compose --version
```

```
2. docker-compose.yml 파일 저장
위 YAML 파일을 현재 작업 디렉토리에 docker-compose.yml로 저장.
```


```
3. Docker 네트워크 설정
docker-compose.yml에서 이미 spark-net이라는 bridge 네트워크를 사용하도록 설정되어 있음.
```

```
4. 컨테이너 실행
터미널에서 docker-compose.yml이 있는 디렉토리로 이동 후 실행:

sh
docker-compose up -d
```

```
컨테이너 상태 확인

sh
docker container -ls
```


컨테이너 목록

spark-master

spark-worker

jupyter

```
5. 실행된 서비스 접속
Spark UI 확인

http://localhost:8080 (Spark Master)

http://localhost:4040 (Spark Job UI)

Jupyter Notebook 접속

http://localhost:8888

실행 후 제공된 토큰을 사용하여 로그인.

Spark Shell 테스트

sh
docker exec -it spark-master /opt/bitnami/spark/bin/spark-shell --master spark://spark-master:7077
실행되면 아래와 같이 확인 가능:

scala
scala> val data = spark.range(1, 100)
scala> data.show()
```


```
6. Jupyter에서 Spark 사용 설정
Jupyter Notebook에서 pyspark 실행을 위해 새 노트북을 열고 아래 코드 실행:

python

from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("Test") \
    .master("spark://spark-master:7077") \
    .getOrCreate()

df = spark.createDataFrame([(1, "Alice"), (2, "Bob")], ["id", "name"])
df.show()
```

```
7. 컨테이너 정리
실행 중인 컨테이너를 중지하고 삭제하려면:

sh

docker-compose down
```

확장

Kafka 추가 → 실시간 데이터 스트리밍 처리

Zookeeper 추가 → Kafka 클러스터 관리

Spark Structured Streaming → Kafka 데이터를 Spark에서 실시간 처리

PostgreSQL 추가 → 처리된 데이터를 저장

데이터 크롤러 (Scrapy or Python script) → Kafka로 데이터 전송



```
version: '3.8'

services:
  spark-master:
    image: bitnami/spark:3.3.2
    container_name: spark-master
    ports:
      - "8080:8080"
      - "4040:4040"
      - "7077:7077"
    environment:
      - SPARK_MODE=master
      - SPARK_MASTER_HOST=spark-master
    networks:
      - spark-net

  spark-worker:
    image: bitnami/spark:3.3.2
    container_name: spark-worker
    environment:
      - SPARK_MODE=worker
      - SPARK_MASTER_URL=spark://spark-master:7077
    depends_on:
      - spark-master
    networks:
      - spark-net

  jupyter:
    image: jupyter/pyspark-notebook
    container_name: jupyter
    ports:
      - "8888:8888"
    depends_on:
      - spark-master
    networks:
      - spark-net

  zookeeper:
    image: bitnami/zookeeper:latest
    container_name: zookeeper
    ports:
      - "2181:2181"
    environment:
      - ALLOW_ANONYMOUS_LOGIN=yes
    networks:
      - spark-net

  kafka:
    image: bitnami/kafka:latest
    container_name: kafka
    ports:
      - "9092:9092"
    environment:
      - KAFKA_BROKER_ID=1
      - KAFKA_ZOOKEEPER_CONNECT=zookeeper:2181
      - KAFKA_ADVERTISED_LISTENERS=PLAINTEXT://kafka:9092
      - ALLOW_PLAINTEXT_LISTENER=yes
    depends_on:
      - zookeeper
    networks:
      - spark-net

  postgres:
    image: postgres:latest
    container_name: postgres
    environment:
      POSTGRES_USER: admin
      POSTGRES_PASSWORD: admin
      POSTGRES_DB: streaming_db
    ports:
      - "5432:5432"
    networks:
      - spark-net

  scraper:
    build: ./scraper
    container_name: scraper
    depends_on:
      - kafka
    networks:
      - spark-net

networks:
  spark-net:
    driver: bridge

```

```
추가된 구성 요소 설명
서비스	설명
zookeeper	Kafka 클러스터 관리를 위한 서비스
kafka	실시간 데이터 스트리밍을 위한 메시지 브로커
postgres	Spark에서 처리한 데이터를 저장하는 데이터베이스
scraper	데이터를 크롤링하고 Kafka에 전송하는 Python 서비스
```


```
scraper/ 디렉토리 생성 후 Dockerfile 추가
scraper/Dockerfile:

dockerfile

FROM python:3.9

WORKDIR /app
COPY requirements.txt .
RUN pip install -r requirements.txt
COPY scraper.py .
CMD ["python", "scraper.py"]
```

```
크롤러 패키지 설치 (scraper/requirements.txt)

txt

requests
beautifulsoup4
kafka-python
```

```
Kafka로 데이터 전송하는 scraper.py 작성 scraper/scraper.py:

python

import time
import requests
from bs4 import BeautifulSoup
from kafka import KafkaProducer

KAFKA_BROKER = "kafka:9092"
TOPIC = "web-data"

producer = KafkaProducer(bootstrap_servers=KAFKA_BROKER)

def fetch_data():
    url = "https://news.ycombinator.com/"
    response = requests.get(url)
    soup = BeautifulSoup(response.text, "html.parser")
    
    articles = []
    for item in soup.select(".storylink"):
        articles.append(item.text)

    return articles

while True:
    data = fetch_data()
    for article in data:
        producer.send(TOPIC, article.encode('utf-8'))
    print("Data sent to Kafka")
    time.sleep(10)
```